In [1]:
import sys
from pathlib import Path
sys.path.append("/kb/module/berdl")
from berdl.genome_paths import GenomePaths
from berdl.prep_genome_set import BERDLPreGenome
from cobrakbase import KBaseAPI

modelseedpy 0.4.2
cobrakbase 0.4.0


In [2]:
from pathlib import Path
paths = GenomePaths(root=Path("/test/syncom").resolve())

In [3]:
kbase = KBaseAPI('JLQJGZGRPPXV2VPTJJM6HFWM4NRK46QW')

In [4]:
genome_R12 = kbase.get_from_ws('158561/3/1')
genome_3H11 = kbase.get_from_ws('158561/6/1')

In [5]:
berdl_prep_genomes = BERDLPreGenome(kbase, paths)

In [6]:
user_to_clade, ani_clade, df_ani_fitness, df_ani_phenotype = berdl_prep_genomes.run([genome_R12, genome_3H11])

In [8]:
ani_clade

{'Rhodanobacter-sp.FW510-R12': {'RS_GCF_021545845.1': [100.0, 100.0, 100.0],
  'RS_GCF_000264375.1': [94.18, 73.35, 70.98],
  'RS_GCF_000230695.2': [90.73, 57.07, 57.36],
  'RS_GCF_003069545.1': [89.92, 61.33, 51.62],
  'RS_GCF_001617365.1': [88.84, 50.09, 44.29],
  'RS_GCF_018599185.1': [88.27, 50.1, 45.62],
  'GB_GCA_001898415.1': [85.46, 20.18, 19.57],
  'RS_GCF_016889805.1': [85.31, 23.64, 22.06]},
 'Acidovorax-sp.GW101-3H11': {'RS_GCF_001633105.1': [99.99, 99.82, 99.21],
  'RS_GCF_002245625.1': [93.76, 81.44, 76.42],
  'RS_GCF_000302535.1': [88.27, 46.25, 43.89],
  'RS_GCF_007993815.1': [88.08, 46.43, 42.58],
  'RS_GCF_023913775.1': [87.81, 32.99, 38.56],
  'RS_GCF_003058055.1': [87.8, 42.48, 39.66],
  'RS_GCF_019218755.1': [86.44, 30.56, 32.14],
  'RS_GCF_000204195.1': [86.33, 32.25, 32.56],
  'RS_GCF_002176815.1': [85.35, 15.89, 12.82],
  'RS_GCF_003669015.1': [85.28, 15.52, 14.3],
  'RS_GCF_001424425.1': [85.23, 22.42, 24.04],
  'GB_GCA_018828125.1': [84.52, 18.75, 17.72]}}

In [7]:
user_to_clade

{'Rhodanobacter-sp.FW510-R12': 'RS_GCF_021545845.1',
 'Acidovorax-sp.GW101-3H11': 'RS_GCF_001633105.1'}

In [10]:
clade_to_user_genomes = {}
for u, c in user_to_clade.items():
    if c not in clade_to_user_genomes:
        clade_to_user_genomes[c] = set()
    clade_to_user_genomes[c].add(u)

In [ ]:
pg_query = QueryPangenomes()

In [13]:
from berdl.berdl_api import BERDLAPI

In [25]:
%%time
berdl_token = "5AROMD2TY337O2E2AGOL52HL5T7EPKT6"
berdl_api = BERDLAPI(berdl_token)
berdl_api.timeout = 12000
genome_ke_member_id = 'GB_GCA_019459165.1'
sql = f"SELECT * FROM kbase_genomes.name WHERE name = '{genome_ke_member_id}' LIMIT 1"
response = berdl_api.sql(sql, offset=0, page_limit=10)

CPU times: user 24.4 ms, sys: 2.24 ms, total: 26.7 ms
Wall time: 55.1 s


In [26]:
response.content

b'{"error":40800,"error_type":"request_timeout","message":"Request timed out after 55.0 seconds. The operation took too long to complete. Consider using pagination, reducing query scope, or breaking up large operations into smaller requests."}'

In [27]:
import polars as pl

In [29]:
!ls /data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_0

contig.parquet
contig_collection.parquet
contig_collection_x_encoded_feature.parquet
contig_collection_x_feature.parquet
contig_collection_x_protein.parquet
contig_x_contig_collection.parquet
contig_x_encoded_feature.parquet
contig_x_feature.parquet
contig_x_protein.parquet
encoded_feature.parquet
encoded_feature_x_feature.parquet
encoded_feature_x_protein.parquet
feature.parquet
feature_name.parquet
feature_prodigal_attributes.parquet
feature_x_protein.parquet
mc_upload_tenant_berdl.sh
mc_upload_user_berdl.sh
name.parquet
protein.parquet


In [39]:
import polars as pl
from pathlib import Path
from berdl.query.query_genome import QueryGenomeABC


class QueryGenomeLocal(QueryGenomeABC):

    def __init__(self):
        self.root_reference = Path('/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes')
        self.ldf_name = pl.scan_parquet(f'{self.root_reference}/block_*/name.parquet')
        pass

    def get_genome_features(self, genome_id: str):
        """Return all gene clusters for a given clade."""
        raise NotImplementedError

    def get_genome_contigs(self, genome_id: str):
        """Return all gene clusters for a given clade."""
        raise NotImplementedError

In [40]:
qg = QueryGenomeLocal()

In [42]:
qg.ldf_name.collect_schema()

Schema([('entity_id', String),
        ('name', String),
        ('description', String),
        ('source', String)])

In [46]:
%%time
df = qg.ldf_name.filter(pl.col("name") == genome_ke_member_id).collect()

CPU times: user 927 ms, sys: 296 ms, total: 1.22 s
Wall time: 51.3 ms


In [50]:
df.height

1

In [53]:
df.row(0, named=True)

{'entity_id': 'CDM:ccol-9034b761-4f23-4bc4-ad6b-54e7ec40366a',
 'name': 'GB_GCA_019459165.1',
 'description': 'gtdb_r214',
 'source': 'ke-pangenomes'}